In [3]:
!pip install --upgrade sagemaker

Keyring is skipped due to an exception: 'keyring.backends'
  Using cached sagemaker-2.129.0-py2.py3-none-any.whl
  Using cached importlib_metadata-4.13.0-py3-none-any.whl (23 kB)
  Using cached boto3-1.26.54-py3-none-any.whl (132 kB)
  Using cached botocore-1.29.54-py3-none-any.whl (10.3 MB)
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 5.1.0
    Uninstalling importlib-metadata-5.1.0:
      Successfully uninstalled importlib-metadata-5.1.0
  Attempting uninstall: botocore
    Found existing installation: botocore 1.29.24
    Uninstalling botocore-1.29.24:
      Successfully uninstalled botocore-1.29.24
  Attempting uninstall: boto3
    Found existing installation: boto3 1.26.24
    Uninstalling boto3-1.26.24:
      Successfully uninstalled boto3-1.26.24
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.120.0
    Uninstalling sagemaker-2.120.0:
      Successfully uninstalled sagemaker-2.120.0
ERROR: pip's depe

In [92]:
import sagemaker
import boto3
import sys
import io
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sagemaker import get_execution_role

In [93]:
prefix = "snowflake-sagemaker-featurestore"
role = get_execution_role()

sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
s3_bucket_name = sagemaker_session.default_bucket()

print(f"Region is {region}")
print(f"IAM role is {role}")

Region is us-east-1
IAM role is arn:aws:iam::240487350066:role/service-role/AmazonSageMaker-ExecutionRole-20210720T164565


In [94]:
sagemaker_session.boto_session.client("sagemaker", region_name=region).list_feature_groups()

{'FeatureGroupSummaries': [{'FeatureGroupName': 'snowflake-dim-place',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:240487350066:feature-group/snowflake-dim-place',
   'CreationTime': datetime.datetime(2023, 1, 23, 19, 25, 39, 544000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created'},
  {'FeatureGroupName': 'snowflake-blog-dw-fs-fact-rating',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:240487350066:feature-group/snowflake-blog-dw-fs-fact-rating',
   'CreationTime': datetime.datetime(2023, 1, 18, 16, 50, 7, 36000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created',
   'OfflineStoreStatus': {'Status': 'Active'}},
  {'FeatureGroupName': 'snowflake-blog-dw-fs-dim-user',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:240487350066:feature-group/snowflake-blog-dw-fs-dim-user',
   'CreationTime': datetime.datetime(2023, 1, 23, 21, 9, 33, 294000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created',
   'OfflineStoreStatus': {'Status': 'Active'}},
  {'FeatureGroupName':

In [95]:
dim_user_feature_group_name = "snowflake-blog-dw-fs-dim-user"
dim_place_feature_group_name = "snowflake-blog-dw-fs-dim-place"
fact_rating_feature_group_name = "snowflake-blog-dw-fs-fact-rating"

In [96]:
from sagemaker.feature_store.feature_group import FeatureGroup

dim_user_feature_group = FeatureGroup(name=dim_user_feature_group_name, sagemaker_session=sagemaker_session)
dim_place_feature_group = FeatureGroup(name=dim_place_feature_group_name, sagemaker_session=sagemaker_session)
fact_rating_feature_group = FeatureGroup(name=fact_rating_feature_group_name, sagemaker_session=sagemaker_session)

In [97]:
# dim_user_feature_group.delete()
# dim_place_feature_group.delete()
# fact_rating_feature_group.delete()

In [98]:
# dim_user_feature_group.describe()

In [99]:
# dim_place_feature_group.describe()

In [100]:
# fact_rating_feature_group.describe()

## Build training dataset

Option 1: Using Feature Store API DatasetBuilder

In [101]:
from sagemaker.feature_store.dataset_builder import DatasetBuilder

fact_rating_dataset = DatasetBuilder(
    sagemaker_session = sagemaker_session, 
    base = fact_rating_feature_group,
    output_path = f"s3://{s3_bucket_name}/{prefix}",
    record_identifier_feature_name = 'ratingID',
    event_time_identifier_feature_name = 'timestamp', 
).to_dataframe()[0]

fact_rating_dataset.head()

,ratingID,userID,placeID,rating_overall,timestamp
0,10556,xb,1034,1,2022-08-25T00:00:00.000Z
1,10627,xj,825,2,2022-08-25T00:00:00.000Z
2,5203,kF,713,2,2022-08-25T00:00:00.000Z
3,7453,pS,542,0,2022-08-25T00:00:00.000Z
4,4550,jf,822,1,2022-08-25T00:00:00.000Z


In [102]:
fact_rating_dataset = fact_rating_dataset.drop(columns=['ratingID', 'timestamp'])

In [103]:
dim_place_dataset = DatasetBuilder(
    sagemaker_session = sagemaker_session, 
    base = dim_place_feature_group,
    output_path = f"s3://{s3_bucket_name}/{prefix}",
    record_identifier_feature_name = 'placeID',
    event_time_identifier_feature_name = 'timestamp', 
).to_dataframe()[0]

dim_place_dataset.head()

,placeID,place_latitude,place_longitude,place_alcohol,place_smoking_area,place_price,place_parking_lot,timestamp
0,132885,22.179517,-100.958436,0.0,0.0,1.0,2.0,2022-08-25T00:00:00.000Z
1,1026,20.940323,-100.736570,0.0,1.0,1.0,1.0,2022-08-25T00:00:00.000Z
2,795,22.260793,-101.028600,0.0,0.0,2.0,0.0,2022-08-25T00:00:00.000Z
3,1250,20.390518,-99.971110,0.0,1.0,1.0,1.0,2022-08-25T00:00:00.000Z
4,135074,22.149689,-100.999525,1.0,2.0,2.0,2.0,2022-08-25T00:00:00.000Z


In [104]:
dim_place_dataset = dim_place_dataset.drop(columns='timestamp')

Option 2: Utilizing the auto-built Glue Data Catalog for FeatureGroup

In [147]:
dim_user_query = dim_user_feature_group.athena_query()
dim_user_table = dim_user_query.table_name

dim_user_query_string = (
    'SELECT * FROM "'
    + dim_user_table
    + '"'
)
print(dim_user_query_string)

SELECT * FROM "snowflake-blog-dw-fs-dim-user-1674508173"


In [148]:
dim_user_query.run(
    query_string = dim_user_query_string,
    output_location = f"s3://{s3_bucket_name}/{prefix}",
)

dim_user_query.wait()
dim_user_dataset = dim_user_query.as_dataframe()

dim_user_dataset.head()

INFO:sagemaker:Query bc16ab54-39b6-48a7-ba88-c23638aca0c8 is being executed.
INFO:sagemaker:Query bc16ab54-39b6-48a7-ba88-c23638aca0c8 successfully executed.


,userid,user_latitude,user_longitude,user_smoker,user_drink_level,user_transport,user_budget,user_interest_technology,user_interest_variety,user_interest_none,user_interest_eco-friendly,user_interest_retro,user_personality_thrifty-protector,user_personality_hard-worker,user_personality_conformist,user_personality_hunter-ostentatious,timestamp,write_time,api_invocation_time,is_deleted
0,jb,21.470053,-101.054680,False,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2022-08-25T00:00:00.000Z,2023-01-23 21:23:38.045,2023-01-23 21:18:42.000,False
1,in,21.114285,-99.067106,False,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2022-08-25T00:00:00.000Z,2023-01-23 21:23:38.045,2023-01-23 21:18:42.000,False
2,zt,23.771030,-100.118181,False,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2022-08-25T00:00:00.000Z,2023-01-23 21:23:38.045,2023-01-23 21:18:42.000,False
3,rP,22.216247,-101.054680,False,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2022-08-25T00:00:00.000Z,2023-01-23 21:23:38.045,2023-01-23 21:18:43.000,False
4,uS,23.771030,-100.745477,False,2.0,2.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2022-08-25T00:00:00.000Z,2023-01-23 21:23:38.045,2023-01-23 21:18:43.000,False


In [107]:
dim_user_dataset = dim_user_dataset.drop(columns = ["timestamp", "write_time", "api_invocation_time", "is_deleted"])

In [108]:
dim_user_dataset.rename(columns = {"userid": "userID"}, inplace=True)
dim_user_dataset.head()

,userID,user_latitude,user_longitude,user_smoker,user_drink_level,user_transport,user_budget,user_interest_technology,user_interest_variety,user_interest_none,user_interest_eco-friendly,user_interest_retro,user_personality_thrifty-protector,user_personality_hard-worker,user_personality_conformist,user_personality_hunter-ostentatious
0,jb,21.470053,-101.054680,False,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,in,21.114285,-99.067106,False,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,zt,23.771030,-100.118181,False,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,rP,22.216247,-101.054680,False,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,uS,23.771030,-100.745477,False,2.0,2.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [109]:
# Merge selected columns from three datasets
final_data = pd.merge(pd.merge(fact_rating_dataset, dim_user_dataset, on='userID'), dim_place_dataset, on='placeID')
final_data.head()

,userID,placeID,rating_overall,user_latitude,user_longitude,user_smoker,user_drink_level,user_transport,user_budget,user_interest_technology,...,user_personality_thrifty-protector,user_personality_hard-worker,user_personality_conformist,user_personality_hunter-ostentatious,place_latitude,place_longitude,place_alcohol,place_smoking_area,place_price,place_parking_lot
0,xb,1034,1,20.082133,-99.872832,False,2.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,23.683763,-99.166705,0.0,0.0,1.0,0.0
1,jL,1034,1,21.063382,-100.449454,False,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,23.683763,-99.166705,0.0,0.0,1.0,0.0
2,jc,1034,0,23.771030,-100.130971,False,0.0,1.0,1.0,0.0,...,1.0,0.0,0.0,0.0,23.683763,-99.166705,0.0,0.0,1.0,0.0
3,hj,1034,0,22.938424,-101.054680,False,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,23.683763,-99.166705,0.0,0.0,1.0,0.0
4,lQ,1034,1,20.576645,-100.386010,False,2.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,23.683763,-99.166705,0.0,0.0,1.0,0.0


In [110]:
final_data.shape

(9609, 24)

In [111]:
# Check data for any nulls
final_data.isnull().values.any()

False

In [112]:
# True/False -> 1/0
final_data['user_smoker'] = final_data['user_smoker'].astype(int)

In [113]:
# Move rating_overall (label) to the first column
first_column = final_data.pop('rating_overall')
final_data.insert(0, 'rating_overall', first_column)

In [114]:
pd.set_option('display.max_columns', None)
final_data.head()

,rating_overall,userID,placeID,user_latitude,user_longitude,user_smoker,user_drink_level,user_transport,user_budget,user_interest_technology,user_interest_variety,user_interest_none,user_interest_eco-friendly,user_interest_retro,user_personality_thrifty-protector,user_personality_hard-worker,user_personality_conformist,user_personality_hunter-ostentatious,place_latitude,place_longitude,place_alcohol,place_smoking_area,place_price,place_parking_lot
0,1,xb,1034,20.082133,-99.872832,0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,23.683763,-99.166705,0.0,0.0,1.0,0.0
1,1,jL,1034,21.063382,-100.449454,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,23.683763,-99.166705,0.0,0.0,1.0,0.0
2,0,jc,1034,23.771030,-100.130971,0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,23.683763,-99.166705,0.0,0.0,1.0,0.0
3,0,hj,1034,22.938424,-101.054680,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,23.683763,-99.166705,0.0,0.0,1.0,0.0
4,1,lQ,1034,20.576645,-100.386010,0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,23.683763,-99.166705,0.0,0.0,1.0,0.0


In [115]:
# Split label column and remove header
feature_id_col = final_data.columns[1:]
label_col = final_data.columns[0]

features_ids = final_data[feature_id_col].values
labels = final_data[label_col].values.astype('float32')

In [119]:
# Split some samples for final testing
X_train_all, X_test, y_train_all, y_test = train_test_split(features_ids, labels, test_size=0.1, stratify=labels)

In [125]:
# Drop userID and placeID columns in training set
X_train_all = X_train_all[:, 2:].astype('float32')

In [128]:
# Split train & validation dataset
X_train, X_validation, y_train, y_validation = train_test_split(X_train_all, y_train_all, test_size=0.2, stratify=y_train_all)

In [129]:
# Prepare data in csv
train_concate = np.concatenate((y_train.reshape(len(y_train),1), X_train), axis=1)
train_data = pd.DataFrame(train_concate, index=None, columns=None)

validation_concate = np.concatenate((y_validation.reshape(len(y_validation),1), X_validation), axis=1)
validation_data = pd.DataFrame(validation_concate, index=None, columns=None)

test_concate = np.concatenate((y_test.reshape(len(y_test),1), X_test), axis=1)
test_data = pd.DataFrame(test_concate, index=None, columns=None)

In [130]:
train_data.to_csv('train_data.csv', header=False, index=False)
validation_data.to_csv('validation_data.csv', header=False, index=False)
test_data.to_csv('test_data.csv', header=False, index=False)

In [131]:
# Upload training and validation data to s3
sagemaker.Session().upload_data('train_data.csv', bucket=s3_bucket_name, key_prefix=prefix+'/train')
sagemaker.Session().upload_data('validation_data.csv', bucket=s3_bucket_name, key_prefix=prefix+'/validation')

's3://sagemaker-us-east-1-240487350066/snowflake-sagemaker-featurestore/validation/validation_data.csv'

In [138]:
# Train XgBoost
container = sagemaker.image_uris.retrieve("xgboost", region, "1.5-1")

hyperparameters = {
    "num_class": "3",
    "max_depth": "9", # 10
    "eta": "0.2",
    "gamma": "4",
    "min_child_weight": "1.3",  # 1.6
    "subsample": "0.7",
    "objective": "multi:softmax",
    "num_round": "254",  # 266
    "verbosity": "2",
    "alpha": "0.07",  # 0.03
    
}

output_path = f"s3://{s3_bucket_name}/{prefix}/output"

xgb_estimator = sagemaker.estimator.Estimator(
    container,
    role,
    base_job_name = "feature-store-xgb",
    hyperparameters = hyperparameters,
    instance_type = "ml.m5.2xlarge",
    instance_count = 1,
    volume_size = 5,
    output_path = output_path,
    sagemaker_session = sagemaker.Session(),
)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [139]:
train_input = sagemaker.inputs.TrainingInput(f"s3://{s3_bucket_name}/{prefix}/train/", content_type="csv")
validation_input = sagemaker.inputs.TrainingInput(f"s3://{s3_bucket_name}/{prefix}/validation/", content_type="csv")

xgb_estimator.fit({'train':train_input, 'validation':validation_input})

INFO:sagemaker:Creating training-job with name: feature-store-xgb-2023-01-23-22-09-44-179


2023-01-23 22:09:44 Starting - Starting the training job......
2023-01-23 22:10:21 Starting - Preparing the instances for training......
2023-01-23 22:11:24 Downloading - Downloading input data...
2023-01-23 22:12:09 Training - Training image download completed. Training in progress...[2023-01-23 22:12:23.501 ip-10-0-238-75.ec2.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-01-23:22:12:23:INFO] Imported framework sagemaker_xgboost_container.training
[2023-01-23:22:12:23:INFO] Failed to parse hyperparameter objective value multi:softmax to Json.
Returning the value itself
[2023-01-23:22:12:23:INFO] No GPUs detected (normal if no gpus installed)
[2023-01-23:22:12:23:INFO] Running XGBoost Sagemaker in algorithm mode
[2023-01-23:22:12:23:INFO] Determined delimiter of CSV input is ','
[2023-01-23:22:12:23:INFO] Determined delimiter of CSV input is ','
[2023-01-23:22:12:23:INFO] files path: /opt/ml/input/data/train
[2023-01-23:22:12:23:INFO] Determined delimiter of CS

In [140]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer

xgb_predictor = xgb_estimator.deploy(
    initial_instance_count=1, 
    instance_type="ml.m5.xlarge",
    serializer=CSVSerializer(),
    deserializer=CSVDeserializer(),
)

INFO:sagemaker:Creating model with name: feature-store-xgb-2023-01-23-22-14-23-072
INFO:sagemaker:Creating endpoint-config with name feature-store-xgb-2023-01-23-22-14-23-072
INFO:sagemaker:Creating endpoint with name feature-store-xgb-2023-01-23-22-14-23-072


-----!

In [141]:
# First three columns in testing data: rating_overall (label column), userID, placeID, thus testing data starts from 4th column
preds = xgb_predictor.predict(test_data.iloc[:,3:].to_numpy())
preds[:5]

[['2.0'], ['1.0'], ['1.0'], ['2.0'], ['1.0']]

In [142]:
acc = accuracy_score(y_test, np.array(preds).flatten().astype(float))
print(f"Accuracy on test data is {acc}")

Accuracy on test data is 0.6607700312174818


In [143]:
xgb_predictor.delete_endpoint(delete_endpoint_config=True)

INFO:sagemaker:Deleting endpoint configuration with name: feature-store-xgb-2023-01-23-22-14-23-072
INFO:sagemaker:Deleting endpoint with name: feature-store-xgb-2023-01-23-22-14-23-072


## Do HPO if needed

In [144]:
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

hyperparameter_ranges = {
    "min_child_weight": ContinuousParameter(1, 5),
    "max_depth": IntegerParameter(1, 10),
    "num_round":IntegerParameter(1, 300),
    "alpha": ContinuousParameter(0, 0.3),
}

objective_metric_name = "validation:accuracy"

tuner = HyperparameterTuner(xgb_estimator, objective_metric_name, hyperparameter_ranges, max_jobs=200, max_parallel_jobs=10)

tuner.fit({'train':train_input, 'validation':validation_input})

INFO:sagemaker:Creating hyperparameter tuning job with name: sagemaker-xgboost-230123-2217


......................................................................................................................................................................................................................................................................................................!


In [145]:
tuner.best_training_job()

'sagemaker-xgboost-230123-2217-070-d8a713b0'

In [146]:
tuner.best_estimator().hyperparameters()


2023-01-23 22:27:57 Starting - Found matching resource for reuse
2023-01-23 22:27:57 Downloading - Downloading input data
2023-01-23 22:27:57 Training - Training image download completed. Training in progress.
2023-01-23 22:27:57 Uploading - Uploading generated training model
2023-01-23 22:27:57 Completed - Resource reused by training job: sagemaker-xgboost-230123-2217-080-db6a9e7d


{'_tuning_objective_metric': 'validation:accuracy',
 'alpha': '0.032823133378856326',
 'eta': '0.2',
 'gamma': '4',
 'max_depth': '10',
 'min_child_weight': '1.62743527216469',
 'num_class': '3',
 'num_round': '266',
 'objective': 'multi:softmax',
 'subsample': '0.7',
 'verbosity': '2'}